In [2]:
# download dataset and store it in s3 bucket
!python ./scripts/download_data.py

In [3]:
# run manual hyperparameter optimization
!python ./scripts/hpo.py 5

2023/08/12 20:55:18 INFO mlflow.tracking.fluent: Experiment with name 'red-wine-quality-hyperopt' does not exist. Creating a new experiment.
[I 2023-08-12 20:55:18,314] A new study created in memory with name: no-name-32ac8e0b-f5e0-41c3-b30d-2a91679645c0
[I 2023-08-12 20:55:19,730] Trial 0 finished with value: 0.645 and parameters: {'n_estimators': 1100, 'max_features': 'sqrt', 'max_depth': 60, 'bootstrap': False, 'min_samples_leaf': 2, 'min_samples_split': 10, 'random_state': 0, 'n_jobs': -1}. Best is trial 0 with value: 0.645.
[I 2023-08-12 20:55:21,401] Trial 1 finished with value: 0.63 and parameters: {'n_estimators': 1950, 'max_features': 'log2', 'max_depth': 60, 'bootstrap': False, 'min_samples_leaf': 1, 'min_samples_split': 2, 'random_state': 0, 'n_jobs': -1}. Best is trial 0 with value: 0.645.
[I 2023-08-12 20:55:21,512] Trial 2 finished with value: 0.66 and parameters: {'n_estimators': 50, 'max_features': 'sqrt', 'max_depth': 90, 'bootstrap': True, 'min_samples_leaf': 2, 'min_

In [1]:
# manually register the best random forest model via prefect flow
!python ./scripts/register_best_model.py

21:25:38.553 | INFO    | prefect.engine - Created flow run 'winged-guillemot' for flow 'Main Registration Flow'
21:25:38.554 | INFO    | Flow run 'winged-guillemot' - View at http://prefect-server:4200/flow-runs/flow-run/4165cf19-beed-4574-a0ce-bd63105f313a
21:25:38.663 | INFO    | Flow run 'winged-guillemot' - Created task run 'Prepare Data-0' for task 'Prepare Data'
21:25:38.664 | INFO    | Flow run 'winged-guillemot' - Executing 'Prepare Data-0' immediately...
21:25:38.788 | INFO    | Task run 'Prepare Data-0' - Finished in state Completed()
21:25:38.803 | INFO    | Flow run 'winged-guillemot' - Created task run 'Get Parameters-0' for task 'Get Parameters'
21:25:38.804 | INFO    | Flow run 'winged-guillemot' - Executing 'Get Parameters-0' immediately...
21:25:38.899 | INFO    | Task run 'Get Parameters-0' - Finished in state Completed()
21:25:38.914 | INFO    | Flow run 'winged-guillemot' - Created task run 'Register Best Model-0' for task 'Register Best Model'
21:25:38.915 | INFO  

In [2]:
# register minio block
!python ./scripts/register_minio_block.py

In [1]:
# register & run a new prefect flow deployment
# you can adjust the cron parameter for the prefered schedule
!prefect deployment build ./scripts/register_best_model.py:main_flow -n rfc-red-wine -p default-agent-pool -q red-wine -sb remote-file-system/minio --cron "0 0 * * *" -o ./scripts/main_flow_deployment.yaml
!prefect deployment apply ./scripts/main_flow_deployment.yaml
!prefect deployment run "Main Registration Flow/rfc-red-wine"

Found flow 'Main Registration Flow'
Deployment YAML created at 
'/home/jovyan/work/scripts/main_flow_deployment.yaml'.
Successfully uploaded 2 files to s3://mlflow-artifacts/
Successfully loaded 'rfc-red-wine'
Deployment 'Main Registration Flow/rfc-red-wine' successfully created with id 
'a4afb2c2-f369-4c05-a313-920f5acd4506'.
View Deployment in UI: 
http://prefect-server:4200/deployments/deployment/a4afb2c2-f369-4c05-a313-920f5a
cd4506

To execute flow runs from this deployment, start an agent that pulls work from 
the 'default-agent-pool' work pool:
$ prefect agent start -p 'default-agent-pool'
Creating flow run for deployment 'Main Registration Flow/rfc-red-wine'...
Created flow run 'hissing-labradoodle'.
└── UUID: c14308f9-1e5d-4eb4-8132-0838fcb7435a
└── Parameters: {}
└── Scheduled start time: 2023-08-12 21:51:36 UTC (now)
└── URL: 
http://prefect-server:4200/flow-runs/flow-run/c14308f9-1e5d-4eb4-8132-0838fcb743
5a
